# 1. Source Code

In [39]:
import glob
import os
import re
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [40]:
# Text 파일의 경로를 지정
txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 저장
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

# Test
print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


In [41]:
# 각 문장을 정제하는 함수를 선언
def preprocess_sentence(sentence):
    # 소문자로 변경 후, 양 쪽의 공백을 제거
    sentence = sentence.lower().strip()
    # 명시된 특수문자의 양쪽에 공백을 추가
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)
    # 두 개 이상의 공백을 하나의 공백으로 변경
    sentence = re.sub(r'[" "]+', " ", sentence)
    # 명시된 문자가 아닐 경우, 공백으로 변경
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)
    # 양 쪽의 공백을 제거
    sentence = sentence.strip()
    # 양 끝에 '<start>', '<end>' 추가
    sentence = '<start> ' + sentence + ' <end>'
    
    return sentence

In [42]:
# corpus 에 길이가 1 ~ 15 인 문장을 정제하여 저장
corpus = []

for s in raw_corpus:
    if len(s) == 0: 
        continue  
    sentence = preprocess_sentence(s)
    
    if len(sentence.split()) > 15:
        continue
        
    corpus.append(sentence)

# Test
print(len(corpus))

156227


In [43]:
# 각 문장을 tokenize 하여 12,000 개의 단어로 tokenizer 에 저장
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,
        filters=' ',
        oov_token="<unk>"
    )
    tokenizer.fit_on_texts(corpus)
    
    # tokenizer 를 활용하여 corpus 를 tensor 로 매핑
    tensor = tokenizer.texts_to_sequences(corpus)
    
    # tensor 의 시퀀스 길이를 동일하게 맞춤 (문장의 끝에 pad(0) 를 추가)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  

    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

# Test
print(tensor.shape)

(156227, 15)


In [44]:
# Test
# 문장의 시작은 2, 문장의 끝은 3, 길이가 부족할 경우 pad(0)
tensor[:5, :10]

array([[   2,   50,    4,   95,  303,   62,   53,    9,  946, 6269],
       [   2,   15, 2971,  872,    5,    8,   11, 5747,    6,  374],
       [   2,   33,    7,   40,   16,  164,  288,   28,  333,    5],
       [   2,   11,  336,   23,   41,    3,    0,    0,    0,    0],
       [   2,    6, 4490,    5,    6, 2040,    3,    0,    0,    0]],
      dtype=int32)

In [45]:
# source_input, target_input 선언
# src_input 은 주로 pad(0) 가 삭제
src_input = tensor[:, :-1]
# tgt_input 은 '<start>'(2) 가 삭제
tgt_input = tensor[:, 1:]

# Test
src_input.shape
tgt_input.shape

(156227, 14)

In [46]:
# train, val dataset 으로 split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input,
                                                          tgt_input,
                                                          test_size=0.2,
                                                          random_state=10)

In [47]:
# Test
print(enc_train.shape)
print(enc_val.shape)
print(dec_train.shape)
print(dec_val.shape)

(124981, 14)
(31246, 14)
(124981, 14)
(31246, 14)


In [48]:
# Train Dataset Object
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 256
steps_per_epoch = len(enc_train) // BATCH_SIZE
# pad(0) 을 포함하여 단어장의 길이는 12,001
VOCAB_SIZE = tokenizer.num_words + 1

# Train Dataset 생성 (enc_train, dec_train)
train_dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
train_dataset = train_dataset.shuffle(BUFFER_SIZE)
# data 를 최대한 확보하기 위해 drop_remainder=False
train_dataset = train_dataset.batch(BATCH_SIZE, drop_remainder=False)

# Test
print(train_dataset)
print(len(train_dataset))

<BatchDataset shapes: ((None, 14), (None, 14)), types: (tf.int32, tf.int32)>
489


In [49]:
# Validation Dataset Object
BUFFER_SIZE = len(enc_val)
steps_per_epoch = len(enc_val) // BATCH_SIZE

# Validation Dataset 생성 (enc_val, dec_val)
val_dataset = tf.data.Dataset.from_tensor_slices((enc_val, dec_val))
val_dataset = val_dataset.shuffle(BUFFER_SIZE)
# data 를 최대한 확보하기 위해 drop_remainder=False
val_dataset = val_dataset.batch(BATCH_SIZE, drop_remainder=False)

# Test
print(val_dataset)
print(len(val_dataset))

<BatchDataset shapes: ((None, 14), (None, 14)), types: (tf.int32, tf.int32)>
123


In [52]:
# Model 을 상속받는 TextGenerator Class 생성
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        # embedding, rnn_1, rnn_2, linear 4 layers
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out

# word vector 의 차원 수
embedding_size = 256
# 일꾼 수
hidden_size = 1800
# model 에 TextGenerator 인스턴스 생성
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [53]:
# 배치 하나를 가져와서 Test
for src_sample, tgt_sample in train_dataset.take(1):
    break
    
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[-7.42763441e-05,  2.14289888e-04, -8.60783475e-05, ...,
          3.93387774e-04, -6.47540655e-05, -7.08866792e-05],
        [-1.21060148e-04,  4.88928810e-04, -1.81302501e-04, ...,
          4.62958240e-04, -5.54173348e-05,  1.26629428e-04],
        [-1.40922741e-04,  6.32188632e-04, -1.47154322e-04, ...,
          3.58747173e-04,  7.91960701e-05,  2.40859910e-04],
        ...,
        [-1.16421969e-03, -5.55926934e-04, -2.62523972e-04, ...,
          1.44930629e-04,  8.51180987e-04,  1.64432020e-03],
        [-1.60559721e-03, -7.98600842e-04, -4.12732043e-04, ...,
         -3.76050361e-04,  1.03018014e-03,  1.96206593e-03],
        [-2.03839503e-03, -1.03370880e-03, -5.67074690e-04, ...,
         -9.23107262e-04,  1.19664101e-03,  2.23791040e-03]],

       [[-7.42763441e-05,  2.14289888e-04, -8.60783475e-05, ...,
          3.93387774e-04, -6.47540655e-05, -7.08866792e-05],
        [ 7.55212895e-05, -4.44176330e-05, -3

In [54]:
# Model.summary()
model.summary()

Model: "text_generator_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      multiple                  3072256   
_________________________________________________________________
lstm_10 (LSTM)               multiple                  14810400  
_________________________________________________________________
lstm_11 (LSTM)               multiple                  25927200  
_________________________________________________________________
dense_5 (Dense)              multiple                  21613801  
Total params: 65,423,657
Trainable params: 65,423,657
Non-trainable params: 0
_________________________________________________________________


In [55]:
# optimizer 선언
optimizer = tf.keras.optimizers.Adam()
# loss algorithm 선언
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

# compile
model.compile(loss=loss, optimizer=optimizer)

# fit (EPOCHS = 10)
model.fit(train_dataset,
          epochs=10,
          validation_data=val_dataset)

Epoch 1/10
489/489 [==============================] - 208s 420ms/step - loss: 3.4141 - val_loss: 2.9995
Epoch 2/10
489/489 [==============================] - 215s 439ms/step - loss: 2.8462 - val_loss: 2.7465
Epoch 3/10
489/489 [==============================] - 215s 440ms/step - loss: 2.5543 - val_loss: 2.5616
Epoch 4/10
489/489 [==============================] - 216s 441ms/step - loss: 2.2704 - val_loss: 2.4293
Epoch 5/10
489/489 [==============================] - 216s 441ms/step - loss: 2.0032 - val_loss: 2.3306
Epoch 6/10
489/489 [==============================] - 216s 442ms/step - loss: 1.7598 - val_loss: 2.2566
Epoch 7/10
489/489 [==============================] - 216s 442ms/step - loss: 1.5472 - val_loss: 2.2072
Epoch 8/10
489/489 [==============================] - 216s 442ms/step - loss: 1.3689 - val_loss: 2.1836
Epoch 9/10
489/489 [==============================] - 216s 442ms/step - loss: 1.2271 - val_loss: 2.1759
Epoch 10/10
489/489 [==============================] - 216s 442m

In [56]:
# 문장 생성 함수 정의
def generate_text(model, tokenizer, init_sentence="<start>", max_len=15):
    # init_sentence 를 tensor 로 변환
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    while True:
        # 입력받은 문장의 tensor 를 모델에 입력
        predict = model(test_tensor) 
        
        # 예측된 값 중 가장 높은 확률의 word index
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        
        # 위에서 예측된 word index를 test_tensor 뒤에 추가
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        
        # 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 종료하고 while 탈출
        if predict_word.numpy()[0] == end_token: 
            break
        if test_tensor.shape[1] >= max_len: 
            break

    generated = ""
    # tokenizer를 활용해 test_tensor 의 index 를 단어로 변환
    # generated 에 저장
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [58]:
# enc_train 으로 학습한 model 의 문장 생성
generate_text(model, tokenizer, init_sentence="<start> i love")

'<start> i love you , liberian girl , <end> '

***

# 2. Retrospection

<h2 style="font-style:italic">2022.01.14 - [E-04]Lyricist Project</h2>

> 이번 프로젝트에서는 팝송의 가사들로부터 학습한 RNN 모델을 활용하여 새로운 가사를 생성하는 Lyricist 를 구현해보았다. 아래에서는 프로젝트를 진행하면서 어려웠던 부분들이나, 알게된 점, 앞으로 학습해나가야할 모호한 점에 대해 회고해보도록 하겠다.

***

**(1)** **어려웠던 점**
<br>
<br>
- 이번 프로젝트에서 가장 어려웠던 점은 적합한 파라미터값을 찾아내는 작업이었다. 이전까지 해왔던 프로젝트와는 달리, 데이터가 많아 학습에 오랜 시간이 걸린 탓에 최적의 파라미터를 찾아내는데 어려움을 겪었다. 다만, 과적합(overfitting)의 개념에 대해 학습한 이후로는 무작정 파라미터의 값을 증가시키는 무모한 실험을 시도하지는 않게 되었다.
<br>
<br>
- 모델에서 세팅한 하이퍼 파라미터는 embedding_size 와 hidden_size 였다. 처음에는 embedding_size 를 256, hidden_size 를 1024 로 세팅하여 학습을 진행했다. 이 때, validation loss 는 2.5 에서 2.6 의 수치가 나왔다. 루브릭 성능 평가 기준은 2.2 이하였으므로, 하이퍼 파라미터를 변경하여야 했고 각 하이퍼 파라미터가 모델의 validation loss 에 어떠한 영향을 미치는지에 대해서 알아보기 위해 embedding_size 를 512, hidden_size 를 2048 로 다시 설정했다. 이 때, loss 는 2.2 보다 낮게 나와주는 듯 했으나, 결국 EPOCH 7 정도부터 증가하는 모습을 보여주었다. 물론, 충분한 EPOCH 를 돌리지 않아 확신할 수는 없었지만, 이를 과적합(overfitting) 으로 생각했다. 다음은 hidden_size 만을 감소시켜보았다. embedding_size 는 512, hidden_size 는 1024 로 실험하였다. loss 는 다시 2.3 ~ 2.4 로 증가하기 시작했고, hidden_size 를 1500 ~ 2000 사이로 fix 하기로 결정을 내렸다. 이후부터는 embedding_size 를 변경시키며 성능을 측정했고, embedding_size 는 생각보다 클 필요가 없다는 점을 깨달았다. 따라서, embedding_size 는 초기값 그대로 256 으로 설정하고, hidden_size 를 1800 으로 시험했다. 물론 마지막 EPOCH 에서는 loss 값이 잠깐 증가하는 양상을 보이나, epoch 10 에 loss 가 한 번 증가했다고 해서 과적합(overfitting) 으로 예상하기에는 무리라고 판단했다. 결국, 2.17 에서 2.18 사이의 2.2 이하 성능 지표값을 도출하였고 이러한 실험 과정에서 모델을 학습시키는 시간이 굉장히 오래걸렸다는 점이 이번 프로젝트에서의 난관이었다.
<br>
<br>
- 정규표현식에 대한 부분도 쉽지 않았다. 사실 정규표현식이 웹크롤링이나 자동화같은 분야에서는 자주 쓰인다는 말을 들어본 적은 있었지만, 이전까지 내가 학습했던 분야들에서는 정규표현식이 필요하지 않았다. 하지만, 다양한 텍스트 파일의 문장에 대해 format 을 전처리해주어야 하는 이번 프로젝트에서 사실상 정규표현식은 모델의 성능에 영향을 미칠 수 있는 중요한 파트였다. 이번 프로젝트를 통해 각 정규표현식에서의 메타 문자와 문법에 대해 맛보았고, 그 중요성에 대해 알게 되었으니 앞으로 꾸준히 호기심을 가지고 학습해나갈 것이다.
<br>
<br>
- RNN 의 원리도 굉장히 복잡하고 어려웠다. embedding layer 와 rnn layer 2개, linear(dense) 의 4 개의 layer 로 구성된 간단한 RNN 이었지만, LSTM 레이어에서 이루어지는 과정들에 대해서 감이 잘 잡히지 않았다. 지금까지 이해한 바로는, embedding layer 에서는 들어온 입력 데이터들의 단어들은 one-hot encoding 처럼 추상화되어 LSTM layer 로 전달된다. 즉, embedding_size 가 극단적으로 클 경우에는 각 단어가 지나치게 추상화되어 모델에 혼란을 줄 수 있게 되는 것이다. 이후, LSTM 층에서는 문장을 순차적으로 읽으며 단어 간 연관성을 분석하게 된다. 마지막으로 이러한 연관성을 기반으로 linear 층에서 다음에 올 단어들을 예측하여 생성하게 된다. 아직까지는 LSTM 에서 하는 역할에 대해 정확하게 설명하기 힘들지만, 앞으로 RNN 의 각 계층에 대해 학습해나가면서 모델을 적절하게 설계하고, 최적의 하이퍼 파라미터 값을 찾아내는 방법을 알아가게 될 것이다.
<br>
<br>

**(2)** **알게된 점**
<br>
<br>
- 정규표현식의 기본적인 메타문자나 그룹핑에 대해 알게 되었다. ^, [], \, - 등등 정규표현식의 메타문자는 내 프로젝트에서 사용할 일이 없다고 하더라도 다른 사람의 코드를 읽어내는데 필요한 지식이다. 또한, ()를 이용하여 정규표현식 내에서 그룹핑을 하게되면, ()안의 형식에 해당하는 문자를 텍스트에서 찾아내고, 이를 r'\1'과 같이 호출하여 사용할 수도 있다. 정규표현식에서의 캡처와 연계되는 개념이므로, 앞으로 캡쳐에 대해 학습해 나갈 생각이다.
<br>
<br>
- 앞서 어려웠던 점에서 설명했었던 것처럼 RNN 각 계층의 단순한 구조에 대해서 알게 되었다. 다시 짧게 설명해보자면, 이번 프로젝트에서 RNN 은 각각 Embedded layer, RNN layer, Linear layer 로 나누어진다. Embedded layer 는 입력된 데이터들을 추상화하는 역할을 수행하게 되며, RNN layer 는 입력된 데이터들을 읽어가며, 각 데이터들의 연관성을 분석해나간다. 마지막으로, linear layer 에서는 앞서 분석한 연관성을 기반으로 새로운 단어들을 예측하여 생성하게 된다. 다른 계층의 이름이나, 역할, 원리 등에 대해서는 차차 학습해나가보기로 한다.
<br>
<br>
- 정제된 각 문장들을 토큰화하고, 텐서로 변환해주는 과정을 구현해보았다. 실제로 컴퓨터에 각 문장들을 그대로 입력하게 되면, 컴퓨터는 이를 읽어낼 수 없다. 각 문장은 단어들로 토큰화되어 각각의 인덱스를 가지고 단어장에 등록되어야 하며, 문장들은 다시 인덱스에 맞게 숫자로 변환되어 표현되어야 한다. 이렇게 한 문장을 단어로 구분하여 각 단어의 인덱스로 표현한 형태를 tensor 라고 한다. 스칼라, 벡터처럼 tensor 는 수치값의 나열이며, 이번 프로젝트에서는 문장을 표현한다. 또한, 이렇게 tensor 로 변환된 각각의 문장들은 서로의 길이를 맞춰주어야 한다. 이는 이후에 있을 연산들을 위해 수행하는 작업으로 사료된다. 이 때, 짧은 길이의 문장들은 그 tensor 의 길이도 짧기 때문에, max_len의 길이에 맞춰 padding을 추가로 할당받는다.
<br>
<br>
- 이외에 모델에 넣을 데이터를 batch 단위로 나누어주는 작업을 수행하기도 했다. 10만개가 넘어가는 모든 문장들을 한 번에 model 에 넣게 되면, 속도와 성능에 영향을 미치게 될 것이다. 따라서, 모델에 한 번에 넣게 될 데이터의 사이즈를 batch_size 로 나누어주고 1 EPOCH 의 1 step 마다 1 batch 를 넣어 모델을 학습시키게 된다. 이렇게 각 batch 가 모두 학습되어 전체 데이터에 대한 학습이 1회 끝나면, 이를 1 EPOCH 라 한다. 이 때, 전체 데이터가 각 batch 에 할당된 데이터의 수로 나누어 떨어지지 않을 경우에는 drop-remainder 옵션을 활용하여, 남은 데이터들을 drop 할 것인지 따로 모아 하나의 batch 로 할당할 것인지를 선택할 수 있다.
<br>
<br>

**(3)** **모호한 점**
<br>
<br>
- 프로젝트의 난이도가 상당히 어렵게 느껴졌던 만큼, 감이 잡히지 않는 부분이 많았다. 예를 들면, '\<start>'와 '\<end>'가 그러했다. 정제된 문장 전체를 x_data 와 y_data, 즉 입력 데이터와 label로 나누어 줄 때, 입력 데이터의 앞에는 '\<start>' 가 붙고 '\<end>' 를 제거해주었으며, 반대로 label 데이터에 대해서는 '\<end>' 를 마지막에 붙이고, 문장의 앞에서 '\<start>' 를 제거해주었다. 입력 데이터의 형식과 label 데이터의 형식이 가지는 이러한 차이가 모델의 학습 및 문장 생성 과정에서 어떠한 의미를 가지는지 더 학습하여야한다.
<br>
<br>
- 이외에 정규표현식에 관해서도 추가적인 학습이 필요할 것이다. 지금까지는 프로젝트를 진행하면서 사용할 일이 없어 학습하지 않았지만, 앞으로 머신러닝 및 인공지능과 관련한 다른 사람의 코드를 접하게 될 때, 용이하게 그 코드를 해석할 수 있기 위해서는 정규표현식에 대한 학습이 필요할 것이다.
<br>
<br>
- 처음 접해본 순환신경망에 대해 학습해나갈 계획이다. 합성곱에 이어 이번에 접한 순환신경망은 그 구조와 원리에 대해 합성곱만큼 알지 못하기 때문에 더욱 어려웠다. 심지어 합성곱을 활용한 이미지 분류기는 설계 중 사진을 확인할 수 있어 직관적으로 진행을 파악할 수 있지만, 텍스트를 활용한 순환신경망 모델이었다보니, 해당 부분에 대해 이해가 더욱 어려웠던 것 같다. 순환신경망의 각 계층과 계층의 역할에 대해 앞으로 자세히 학습해나갈 예정이다.